In [1]:
import scvi
import scanpy as sc
import anndata
import skmisc

In [30]:
# adata = scvi.data.read_loom(filename='/home/ec2-user/scrnaseq_tcsl154/scrna.loom', sparse=True)
# from scipy.sparse import csr_matrix
# adata.X = csr_matrix(adata.X)
# adata.write('/home/ec2-user/scrnaseq_tcsl154/scrna.h5ad')

In [2]:
adata = anndata.read('/home/ec2-user/scrnaseq_tcsl154/scrna.h5ad')
sc.pp.filter_genes(adata, min_counts=3)

/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [3]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state in `.raw`

In [4]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    subset=True,
    layer="counts",
    flavor="seurat_v3"
)

/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True
/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [21]:
scvi.data.setup_anndata(adata, labels_key='ClusterID', layer="counts")

INFO      No batch_key inputted, assuming all cells are same batch                           
INFO      Using labels from adata.obs["ClusterID"]                                           
INFO      Using data from adata.layers["counts"]                                             
INFO      Computing library size prior per batch                                             
INFO      Successfully registered anndata object containing 44627 cells, 87 genes, 1 batches,
          17 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0    
          extra continuous covariates.                                                       
INFO      Please do not further modify adata until model is trained.                         


/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [42]:
import numpy as np

In [43]:
np.unique(adata.obs.ClusterID)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
      dtype=int32)

In [31]:
model.save("scvi_model/")

In [27]:
model = scvi.model.SCVI.load(adata=adata, dir_path="scvi_model/", use_cuda=True)

In [28]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent